#  causality sentence Identifier

## 1.  load sentence

In [19]:
import ast
import pandas as pd
#'data/in/semeval/semeval_causalityDetection.csv'
#'data/in/SCITE/SCITE_TrueFalse.csv'
#'data/in/RAG/ADE/ADE_causalityTEST.csv'
#load semeval data with embed

pm_DB = pd.read_csv('data/in/KNN_RAG/semeval_causalityDetection_cleaned.csv')
instance_emb = pm_DB['embedings'].to_list()
instance_emb = [ast.literal_eval(item) for item in instance_emb]

instances = pm_DB['sentence'].to_list()

len(instances)

549

In [20]:
pm_DB['sentence'][2]

'The deleveraging triggered by the crisis implies that countries that hoarded reserves have been reaping the benefits.'

In [21]:
###few shot example retriever.
# 1. if pattern matched give matched examples as few shot. else give random 10 examples

input_sentence = pm_DB['sentence'][0]
input_sentence

'Dogs develop a fever from stress and/or pain such as in a severe flea infestation.'

## 2. load fewshot database

In [22]:
#load semeval sentence and patterns.
semeval_data = pd.read_csv('data/in/RAG/comb_SCITEsemADE_CausalityPattern.csv')# 

semeval_data = semeval_data.dropna(subset=['causality_phrase'])
len(semeval_data)

#semeval_data = pd.read_csv('data/in/semeval_CE_pattern.csv')# data/in/ADE/ADE_CE_pattern.csv
causality_phrase = semeval_data['causality_phrase'].to_list()
len(causality_phrase)

2365

## 3. get causality label for sentence

In [23]:
#pattern matcher function.
import random
from fuzzywuzzy import fuzz

def get_fewshot_examples_index(input_sentence) :

    #get few shot example index using Leiveinstein ratio.

    fewshot_index = []
    for ind in range (len(causality_phrase)):
        p = causality_phrase[ind]
        window_size = len(p)

        # Slide through input_sentence with a window size of 8
        for i in range(len(input_sentence) - window_size + 1):
            window = input_sentence[i:i + window_size]
            score = fuzz.ratio(window, p)
            if score >90:
                fewshot_index.append(ind)
                #print('p is ',p, 'i is', i)
                #print(f"Window '{window}' - Levenshtein Score: {score}")
    fewshot_index = list(set(fewshot_index))
    
    if len(fewshot_index)>10:
        fewshot_index = random.sample(fewshot_index, 10)
        #fewshot_index= fewshot_index[:10]
        
    
    return fewshot_index
    

#get all sentences and save as json example1

In [24]:

def get_fewshotexamples(fewshot_index):


    train_sentence = semeval_data['sentence'].to_list()

    fewshot_examples = [train_sentence[i] for i in fewshot_index]

    import json
    fewshot_examples = {f'example {i + 1}': sentence for i, sentence in enumerate(fewshot_examples)}
    fewshot_examples = json.dumps(fewshot_examples)
    #print(fewshot_examples)
    
    return fewshot_examples



In [25]:
def get_randomfewshotexamples():
    train_sentence = semeval_data['sentence'].to_list()

    rand_index = [1, 5, 8, 10, 30, 100, 102, 105, 109, 120]
    
    #fewshot_examples = [train_sentence[i] for i in range(10)]
    fewshot_examples = [train_sentence[i] for i in rand_index]

    import json
    fewshot_examples = {f'example {i + 1}': sentence for i, sentence in enumerate(fewshot_examples)}
    fewshot_examples = json.dumps(fewshot_examples)
    #print(fewshot_examples)
    
    return fewshot_examples
    
    

In [26]:
fewshot_index = get_fewshot_examples_index(input_sentence)
fewshot_examples = get_fewshotexamples(fewshot_index)    

In [27]:
import groq
from groq import Groq

client = groq.Groq(api_key= "enter your API key here..")#


In [28]:
#gemma2-9b-it# llama3-8b-8192
def CS_Identifier(causality_sentence, fewshot_examples):

    chat_completion = client.chat.completions.create(
    #
    # Required parameters
    #
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
            "role": "system",
            "content": """
            You will be given a sentence, and your task is to identify whether the sentence shows a 
          causality relation or not. If the sentence shows a causality relation, you can output 1; if it doesn't, output 0. 
          Your output should be in json format: {output :  }.
          
          Hint: If the sentence shows causality relation, you will be able to identify a pair of 
          phrase in the sentence where one phrase will be the cause of the other phrase(effect)
          
         Below are 10 example sentences which shows causality relation  with  their cause and effect 
          phrase delimited between the XML tags <cause>, </cause> , <effect> and </effect>.
           """+ fewshot_examples 
        },
        {
          "role": "user",
          "content": input_sentence 
        },
    ],

    # The language model which will generate the completion.
    model="mixtral-8x7b-32768",
    temperature=0.0,
    seed = 1,
    max_tokens=100,
    top_p=1,
    response_format={"type": "json_object"},)
    out = chat_completion.choices[0].message.content
    
    return out

In [29]:
import json

causality_identifier=[]
fewshotfound_inputsentence_index=[]
for i in range (len(pm_DB)):
    input_sentence = pm_DB['sentence'][i]
    # few shot examples augmentation.
    fewshot_index = get_fewshot_examples_index(input_sentence)############FEWSHOT
    if len(fewshot_index) == 0:
        fewshot_examples =  get_randomfewshotexamples()
    else:
        fewshotfound_inputsentence_index.append(i)
        fewshot_examples = get_fewshotexamples(fewshot_index) #############fewshot
        #fewshot_examples =  get_randomfewshotexamples()##################random shot/zero shot
        
    #print('input_sentence : ', input_sentence)

    print('fewshot_index : ', fewshot_index)
    
    try:
        # output generation
        out = CS_Identifier(input_sentence, fewshot_examples)
        print('out', out)

    
        json_out = json.loads(out)
        causality_identifier_out = json_out['output']
        causality_identifier.append(int(causality_identifier_out))
        
    except TypeError as e:
        causality_identifier.append(0)
        continue
        
    except KeyError as e:
        causality_identifier.append(0)
        continue
    
    except groq.APIConnectionError as e:
        causality_identifier.append(0)
        continue
    except groq.RateLimitError as e:
        print('####################################ratelimit')
        causality_identifier.append(0)
        continue
    except groq.APIStatusError as e:
        causality_identifier.append(0)
        continue
    
    
    print(i)
    print(causality_identifier_out)
    
    

fewshot_index :  [992, 1532, 993, 1001, 1137, 998, 1529, 1551, 997, 1136]
out {"output": 1}
0
1
fewshot_index :  [2093, 2085, 2089, 2092, 279, 1136, 2091, 2086, 278, 1241]
out {"output": 0}
1
0
fewshot_index :  [2158, 481, 2163, 1136, 478, 2161, 482, 2157, 2160, 480]
out {"output": 1}
2
1
fewshot_index :  [1242, 616, 279, 534, 620, 278, 613, 520, 517, 1525]
fewshot_index :  [1136, 1137]
out {"output": 1, "cause": "the researchers placed the compound in a tube", "effect": "the tube was put inside a magnet" }
4
1
fewshot_index :  [2094, 2091, 2090, 2087, 1241, 1242, 2092, 2085, 2086, 2089]
out {"output": 0}
5
0
fewshot_index :  [1528, 986, 1531, 1530, 1242, 2085, 2092, 1527, 2087, 2094]
out {"output": 0}
6
0
fewshot_index :  [482, 1509, 523, 595, 592, 597, 517, 1512, 1533, 552]
out {
"output": 1
}
7
1
fewshot_index :  [278, 1123, 2087, 1527, 1532, 2085, 280, 2090, 279, 2086]
out {
"output": 1
}
8
1
fewshot_index :  [2093, 2085, 2090, 2087, 1242, 2091, 1241, 1136, 2089, 2088]
out {"output

fewshot_index :  [482, 2091, 478, 2087, 1508, 1506, 477, 829, 828, 822]
out {
"output": 0
}
70
0
fewshot_index :  [552, 1075, 548, 520, 521, 523, 2012, 2000, 608, 2003]
out {
"output": 1
}
71
1
fewshot_index :  [1551, 1529, 127, 1527, 996, 1241, 1013, 1014, 123, 998]
out {"output": 0}
72
0
fewshot_index :  [992, 998, 278, 995, 996, 999, 1241, 1001, 1136, 994]
out {"output": 1}
73
1
fewshot_index :  [1506, 1136, 1508, 1510, 1514, 1551, 1515, 1511, 1509, 1513]
out {"output": 0}
74
0
fewshot_index :  [2085, 2091, 1533, 1136, 985, 2092, 2094, 1551, 2093, 1527]
out {"output": "0"}
75
0
fewshot_index :  [909, 595, 1530, 551, 2093, 2087, 516, 985, 549, 2091]
out {"output": 0}
76
0
fewshot_index :  [2088, 1241, 1136, 2092, 2087, 1242, 2090, 2091, 2089, 2094]
out {
"output": 0
}
77
0
fewshot_index :  [514, 1265, 477, 2087, 481, 1507, 552, 2094, 548, 1264]
out {"output": 1}
78
1
fewshot_index :  [2336, 1241, 1506, 1551, 2340, 1531, 1960, 2338, 2337, 1242]
out {"output": 0}
79
0
fewshot_index :  

out {"output": 0}
140
0
fewshot_index :  [1529, 2153, 1530, 1532, 2162, 1527, 2163, 2157, 1136, 2164]
out {"output": 1}
141
1
fewshot_index :  [563, 946, 954, 1533, 514, 953, 566, 947, 1513, 523]
out {"output": 1}
142
1
fewshot_index :  [2090, 1515, 1510, 2088, 1136, 1514, 2093, 2092, 2087, 1513]
out {"output": 1}
143
1
fewshot_index :  [1510, 1513, 1136, 1527, 1515, 1528, 1512, 1506, 1509, 1531]
out {"output": 1}
144
1
fewshot_index :  [995, 554, 519, 2205, 590, 1241, 994, 551, 514, 520]
out {
"output": 1
}
145
1
fewshot_index :  [2089, 1083, 2090, 1514, 1527, 518, 522, 550, 514, 597]
out {"output": 1, "cause": "the escalating cost of funding \"copper bottomed\" final salary pension schemes still provided by councils and other public sector employers such as Transport for London", "effect": "the pension crisis"}
146
1
fewshot_index :  [1241, 1508, 1515, 1512, 1511, 1533, 1513, 1530, 279, 1514]
out {"output": 0}
147
0
fewshot_index :  [514, 1258, 591, 516, 1263, 480, 553, 521, 479, 126

out {
"output": 1
}
206
1
fewshot_index :  [616, 520, 517, 519, 1532, 516, 1533, 614, 619, 521]
out {"output": 1, "cause": "Accident", "effect": "delays on BW Parkway"}
207
1
fewshot_index :  [1242, 1260, 550, 551, 482, 1261, 521, 1258, 553, 478]
out {"output": 1, "cause": "irritants like fabrics, hair gels and makeup", "effect": "Acne"}
208
1
fewshot_index :  [1528, 2088, 515, 2086, 523, 522, 279, 280, 516, 1527]
out {"output": 1}
209
1
fewshot_index :  [1506, 2094, 2087, 998, 2093, 1507, 2092, 992, 995, 1001]
out {
"output": 1
}
210
1
fewshot_index :  [1136, 997, 994, 2341, 278, 993, 2337, 998, 2334, 2332]
out {"output": 1, "cause": "wax producing workers", "effect": "workers build the comb with it"}
211
1
fewshot_index :  [2093, 2088, 2090, 1530, 2087, 2085, 2089, 1531, 1528, 1529]
out {"output": 0}
212
0
fewshot_index :  [2086, 1515, 1532, 1529, 1513, 2093, 1531, 1511, 2089, 1507]
out {"output": 0}
213
0
fewshot_index :  [1507, 2091, 986, 1713, 479, 2089, 591, 514, 594, 1533]
out {

out {"output": "0"}
274
0
fewshot_index :  [2087, 2093, 1527, 2086, 2091, 2092, 2094, 1136, 1529, 1528]
fewshot_index :  [1529, 2091, 2085, 1531, 2092, 2093, 1533, 1532, 2094, 2088]
out {
"output": 1
}
276
1
fewshot_index :  [2089, 2094, 2332, 2333, 2085, 1528, 2093, 2339, 1532, 1531]
out {"output": 0}
277
0
fewshot_index :  [1530, 2334, 521, 520, 550, 514, 592, 2338, 1532, 555]
out {
  "output": 1
}
278
1
fewshot_index :  [478, 1528, 477, 1508, 1241, 1515, 1513, 1529, 1532, 1243]
out {"output": 0}
279
0
fewshot_index :  [521, 522, 520, 1551, 519, 517, 514, 516, 1136, 515]
out {"output": 1, "cause": "microcracks", "effect": "stress or fragility fractures"}
280
1
fewshot_index :  [2093, 2090, 2091, 2092, 1533, 1530, 1528, 1242, 1531, 2088]
out {"output": 1}
281
1
fewshot_index :  [280, 521, 2085, 1528, 1530, 1527, 2091, 522, 1531, 279]
out {"output": 1, "cause_phrase": "Most illnesses, including colds and flu", "effect_phrase": "a toxic overload that also increases the stress on the kid

out {"output": 1, "cause": "the Israeli blockade", "effect": "the crisis"}
344
1
fewshot_index :  [1532, 2089, 1527, 1136, 2086, 2093, 1529, 2092, 2088, 1530]
out {"output": 0}
345
0
fewshot_index :  [552, 556, 1511, 1529, 986, 480, 1530, 594, 549, 555]
out {"output": 1, "cause": "cosmetics", "effect": "this mild and relatively common form of acne"}
346
1
fewshot_index :  [2078, 2082, 1530, 1242, 479, 1241, 1268, 1529, 1551, 482]
out {"output": 0}
347
0
fewshot_index :  [1527, 1528, 1529, 1530, 1531, 1532, 1533]
out {"output": 0}
348
0
fewshot_index :  [1512, 1510, 1509, 1506, 1514, 1241, 1515, 1242, 1508, 1511]
out {"output": 0}
349
0
fewshot_index :  [2086, 2093, 2091, 2094, 2085, 2087, 2092, 1136, 2088, 2090]
out {"output": 0}
350
0
fewshot_index :  [1531, 1506, 1532, 1508, 1509, 1533, 1761, 1762, 1507, 1514]
out {"output": 0}
351
0
fewshot_index :  [1532, 1511, 1509, 1533, 1531, 1530, 1512, 1508, 1528, 1510]
out {"output": 1}
352
1
fewshot_index :  [321, 557, 1532, 441, 444, 2088, 

out {"output": "0"}
408
0
fewshot_index :  [2085, 1551, 2091, 2092, 2089, 2090, 2087, 2086, 2093, 1136]
out {"output": 0}
409
0
fewshot_index :  [1514, 1513, 1000, 1508, 1527, 998, 1511, 1528, 1001, 1242]
out {"output": 1, "cause": "chronic coughing from smoking", "effect": "increases the risk of developing a hernia"}
410
1
fewshot_index :  [594, 518, 2204, 523, 1257, 515, 481, 1260, 1261, 519]
out {
"example 1": 1,
"example 2": 1,
"example 3": 1,
"example 4": 1,
"example 5": 1,
"example 6": 1,
"example 7": 1,
"example 8": 1,
"example 9": 1,
"example 10": 1
}
fewshot_index :  [1136, 1527, 1528, 1529, 1530, 1531, 1532, 1533]
out {"output": 0}
412
0
fewshot_index :  [2341, 1533, 1531, 520, 516, 2335, 1551, 279, 514, 2332]
out {"output": 1, "cause": "most cleaners", "effect": "cause rash or a burning sensation"}
413
1
fewshot_index :  [1513, 1551, 1508, 1511, 1512, 1510, 1514, 1506, 1515, 1136]
out {"output": 0}
414
0
fewshot_index :  [550, 1527, 1508, 2088, 1533, 597, 2091, 317, 1510, 59

out {"output": 0}
475
0
fewshot_index :  [1532, 1533, 1510, 1515, 1509, 1518, 1528, 1513, 1527, 1671]
out {"output": "0"}
476
0
fewshot_index :  [523, 548, 516, 1530, 1511, 550, 518, 562, 521, 1515]
out {
"output": 1
}
477
1
fewshot_index :  [516, 522, 1529, 595, 322, 326, 321, 324, 318, 514]
out {"output": 1}
478
1
fewshot_index :  [2090, 2093, 2086, 1551, 279, 2087, 2092, 280, 2089, 1136]
out {"output": 0}
479
0
fewshot_index :  [2088, 2091, 2336, 2092, 2333, 2339, 2338, 2085, 2335, 2086]
out {"output": 0}
480
0
fewshot_index :  [1372, 592, 557, 522, 590, 323, 1551, 594, 598, 552]
out {"output": 1}
481
1
fewshot_index :  [1531, 279, 1528, 1513, 687, 1514, 1530, 1532, 280, 1529]
out {"output": 0}
482
0
fewshot_index :  [2088, 2092, 1507, 2091, 1512, 1515, 986, 1518, 2085, 2087]
out {"output": 0}
483
0
fewshot_index :  [2337, 1529, 557, 1515, 1514, 1513, 1261, 551, 1509, 1136]
out {"output": 0}
484
0
fewshot_index :  [517, 563, 516, 1507, 1508, 1510, 565, 2091, 521, 2089]
out {"output"

out {"output": 0}
546
0
fewshot_index :  [278, 1510, 1241, 1518, 997, 1528, 1047, 482, 1531, 992]
out {"output": 0}
547
0
fewshot_index :  [278, 125, 1015, 1514, 1013, 1136, 123, 2157, 1509, 478]
out {
"output": 1
}
548
1


In [30]:
len(causality_identifier)

549

In [31]:
len(causality_identifier)

549

In [32]:
len(fewshotfound_inputsentence_index)

547

In [33]:
#get  pattern_matched label
pattern_matched = [0]*(len(pm_DB))

# Update values to 1 at specified indexes
for index in fewshotfound_inputsentence_index:
    if index < len(pattern_matched):
        pattern_matched[index] = 1
pattern_matched

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [34]:
pm_DB['GPT_causality_label'] = causality_identifier
pm_DB['patternmatcher_label'] = pattern_matched
pm_DB

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sentence,label,embedings,GPT_causality_label,patternmatcher_label
0,0,0,0,Dogs develop a fever from stress and/or pain s...,1,"[-0.0036133944522589445, 0.0012565960641950369...",1,1
1,1,1,1,The new pencil and graphite drawing is the pai...,0,"[-0.02040475234389305, 0.022646330296993256, 0...",0,1
2,2,2,2,The deleveraging triggered by the crisis impli...,1,"[-0.02002304047346115, -0.031225474551320076, ...",1,1
3,3,3,3,The fat and cholesterol cause heart disease; t...,1,"[0.002320877043530345, -0.014957467094063759, ...",0,1
4,4,4,4,"The researchers placed the compound in a tube,...",0,"[-0.019478240981698036, 0.0026719982270151377,...",1,1
...,...,...,...,...,...,...,...,...
544,544,621,621,Neuralgia (TN) is the most common cause of chr...,1,"[-0.010079591535031796, 0.014008276164531708, ...",1,1
545,545,622,622,Muscle fatigue is the number one cause of arm ...,1,"[-0.011783338151872158, -0.007658560294657946,...",1,1
546,546,623,623,The movie featured a mutated sheep as the titu...,0,"[-0.015397767536342144, -0.01711292937397957, ...",0,1
547,547,624,624,A plasma was generated by an electric discharg...,1,"[-0.011884856037795544, 0.02052135579288006, 0...",0,1


In [35]:
# Function to set 'prediction' column based on conditions
def set_prediction(row):
    if row['GPT_causality_label'] == 1 and row['patternmatcher_label'] == 1:
        return 1
    else:
        return 0

# Apply the function to each row and create the 'prediction' column
pm_DB['prediction'] = pm_DB.apply(set_prediction, axis=1)
pm_DB

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sentence,label,embedings,GPT_causality_label,patternmatcher_label,prediction
0,0,0,0,Dogs develop a fever from stress and/or pain s...,1,"[-0.0036133944522589445, 0.0012565960641950369...",1,1,1
1,1,1,1,The new pencil and graphite drawing is the pai...,0,"[-0.02040475234389305, 0.022646330296993256, 0...",0,1,0
2,2,2,2,The deleveraging triggered by the crisis impli...,1,"[-0.02002304047346115, -0.031225474551320076, ...",1,1,1
3,3,3,3,The fat and cholesterol cause heart disease; t...,1,"[0.002320877043530345, -0.014957467094063759, ...",0,1,0
4,4,4,4,"The researchers placed the compound in a tube,...",0,"[-0.019478240981698036, 0.0026719982270151377,...",1,1,1
...,...,...,...,...,...,...,...,...,...
544,544,621,621,Neuralgia (TN) is the most common cause of chr...,1,"[-0.010079591535031796, 0.014008276164531708, ...",1,1,1
545,545,622,622,Muscle fatigue is the number one cause of arm ...,1,"[-0.011783338151872158, -0.007658560294657946,...",1,1,1
546,546,623,623,The movie featured a mutated sheep as the titu...,0,"[-0.015397767536342144, -0.01711292937397957, ...",0,1,0
547,547,624,624,A plasma was generated by an electric discharg...,1,"[-0.011884856037795544, 0.02052135579288006, 0...",0,1,0


In [36]:
#get accuracy for fewshot_semeval_causalityDetection: b/w label and GPT_causality_label 

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score

# Calculate precision, recall, and F1 score
precision = precision_score(pm_DB['label'], pm_DB['GPT_causality_label'])
recall = recall_score(pm_DB['label'], pm_DB['GPT_causality_label'])
f1 = f1_score(pm_DB['label'], pm_DB['GPT_causality_label'])
accuracy = accuracy_score(pm_DB['label'], pm_DB['GPT_causality_label'])

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Accuracy: 0.825136612021858
F1 Score: 0.8146718146718147
Precision: 0.7843866171003717
Recall: 0.8473895582329317


In [ ]:

#filtered_rows = pm_DB[(pm_DB['GPT_causality_label'] == 1) & (pm_DB['patternmatcher_label'] == 1)]
#filtered_rows.to_csv('data/out/alzeimers_causalityrelation.csv')

#pm_DB.to_csv('data/out/RAG_causality/semeval/fewshotcomb_semeval_causalityDetection.csv')

In [ ]:
pm_DB.to_csv('data/out/RAG_causality/task1/gemma/zero_SCITE_causalityDetection.csv')#RAG, rand,zero, ADE, SCITE, sem